In [1]:
import requests
import json
from math import ceil
from tqdm import tqdm

import time

/Users/sulab/Documents/pending.api/.venv/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
size = requests.get("https://clinicaltrials.gov/api/v2/stats/size")
total_studies = json.loads(size.content)['totalStudies']

In [4]:
aggregated_studies = []
nextPage = None

request_delay = 1 / 3  #  <= 3 Requests per second

for _ in tqdm(range(ceil(total_studies/1000))):
    try:
        payload = {'format': 'json', 'pageSize': '1000', 'pageToken' : f'{nextPage}'} if nextPage else {'format': 'json', 'pageSize': '1000'}
        data = requests.get("https://clinicaltrials.gov/api/v2/studies", params=payload)
        studies = json.loads(data.content)

        aggregated_studies.extend(studies['studies'])

        # The last page does not have a nextPageToken field
        if 'nextPageToken' not in studies:
            break

        nextPage = studies['nextPageToken']

        time.sleep(request_delay)
    except:
        print('Page Token:', nextPage)
        print('Payload:', payload)
        print('# Aggregated Studies:', len(aggregated_studies))

  0%|          | 0/472 [00:00<?, ?it/s]

100%|█████████▉| 471/472 [24:37<00:03,  3.14s/it]  


In [31]:
print('Expected Studies:', total_studies)
print('Gathered Studies:', len(aggregated_studies))

Expected Studies: 471791
Gathered Studies: 471791


In [32]:
version = requests.get("https://clinicaltrials.gov/api/v2/version").json()
print('Data Time Stamp:', version['dataTimestamp'].split('T')[0])

Data Time Stamp: 2023-11-02
